<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/fasttext_for_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output 

In [2]:
!pip install mwparserfromhell==0.6.4 multiprocess==0.70.13
clear_output()

In [2]:
!pip install fasttext
!pip install sentencepiece
!pip install datasets
clear_output()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


NameError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

dataset_simple = load_dataset("olm/wikipedia", language="simple", date="20230320")

In [ ]:
dataset_simple

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 228181
    })
})

In [ ]:
dataset_simple.save_to_disk('/content/drive/MyDrive/DataSet/simpleenWikiDataset.hfd')

In [5]:
from datasets import load_from_disk
dataset_simple = load_from_disk('/content/drive/MyDrive/DataSet/simpleenWikiDataset.hfd')

In [8]:
newdata = dataset_simple['train']['text'][::100]

In [9]:
len(newdata)

2282

In [10]:
with open("data.txt","w") as wr:
    for item in newdata:
        wr.write("\t".join(item.split(" ")) + "\n")

In [ ]:
from datasets import load_dataset
dataset_fa = load_dataset("wiki_summary")
newdata = dataset_fa['train']['article']#[::50]
with open("datafa.txt","w") as wr:
    for item in newdata:
        wr.write("\t".join(item.split(" ")) + "\n")

Generating train split:   0%|          | 0/45654 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5638 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5074 [00:00<?, ? examples/s]

Dataset wiki_summary downloaded and prepared to /root/.cache/huggingface/datasets/wiki_summary/default/1.1.0/46ede6c8744d6f0c3cc2ec37f2dd1fdfaafc8b97cbf829f7e4529999c81cc0ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Python program to merge two files 
data = data2 = "" 
  
# Reading data from first file 
with open('data.txt') as fp: 
    data = fp.read() 
with open('datafa.txt') as fp: 
    data2 = fp.read() 
# Merging two files into one another file 
data += "\n"
data += data2 
with open ('/content/drive/MyDrive/DataSet/fasttext/finaltext.txt', 'w') as fp: 
  fp.write(data)

In [13]:
import gc

del newdata
#del data2
#del dataset_simple


gc.collect()

240

In [ ]:
import fasttext
model = fasttext.train_unsupervised('data.txt', model='cbow',dim=2048)

In [ ]:
.mmodeel model.save_model('/content/drive/MyDrive/DataSet/myOwnFasttext/fasttext_simple_allwki_dim2048_cbow.model')

In [3]:
import fasttext
model =  fasttext.load_model('/content/drive/MyDrive/DataSet/myOwnFasttext/fasttext_simple_allwki_dim2048_cbow.model')

In [4]:
len(model.words)

14069

In [6]:
import pandas as pd
words = model.get_words()
words_df = pd.DataFrame(words, columns=['token'])
words_df['ftindex'] = words_df.index
words_df

,token,ftindex
0,</s>,0
1,در,1
2,و,2
3,.,3
4,،,4
...,...,...
14064,"House,",14064
14065,Sai,14065
14066,هد,14066
14067,شناورهای,14067


In [7]:
import pandas as pd
llama_vocab = pd.read_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')
llama_vocab.rename(columns = {'Unnamed: 0':'llamaids'}, inplace = True)

In [8]:
vocab_my_merge = pd.merge(llama_vocab,words_df, how='inner', on='token')
vocab_my_merge = vocab_my_merge[['llamaids','ftindex','token']]
vocab_my_merge

,llamaids,ftindex,token
0,2,0,</s>
1,262,13,in
2,264,5205,en
3,265,39,on
4,271,64,at
...,...,...,...
1203,30360,6844,ص
1204,30422,8747,‏
1205,30726,5265,آ
1206,30839,10073,ث


In [ ]:
vocab_my_merge.to_excel('/content/drive/MyDrive/DataSet/myOwnFasttext/mymergewithllama.xlsx')